In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import scipy.stats as stats
warnings.filterwarnings(action='ignore')

In [3]:
# 데이터프레임 불러오기
data = pd.read_csv('/content/drive/MyDrive/Contest/공공조달/dataset/150_data_2.csv', index_col = 0)
data.fillna(data.mean(),inplace=True)
data.columns = ['Green_CNT','Gas','Water','Energy','Waste','Green_AMT','Energy_pr','Gas_reduce','Female_exe',
                'Disabled_pr','Small_CNT','Disabled_CNT','Env','Donation','Member','Female_member','Auditor','Female_RTO','Target' ]
# EDA에서 확인한 이상치 치환
def cut_outlier(df, columns):
    df=df.copy()
    for column in columns:
        q1=df[column].quantile(.25)
        q3=df[column].quantile(.75)
        iqr=q3-q1
        low=q1-1.5*iqr
        high=q3+1.5*iqr
        df.loc[df[column]<low, column]=low
        df.loc[df[column]>high, column]=high
    return df

data = cut_outlier(data, ['Waste', 'Energy'])

In [5]:
# OLS 회귀분석
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from scipy import stats

y_data = data['Target']
X_data = data[['Green_CNT','Green_AMT', 'Small_CNT','Member','Female_member']]
# X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.2, random_state=2022)

art_model = sm.OLS(y_data, X_data)
art_result = art_model.fit()
art_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 Target   R-squared (uncentered):                   0.757
Model:                            OLS   Adj. R-squared (uncentered):              0.748
Method:                 Least Squares   F-statistic:                              89.51
Date:                Thu, 17 Feb 2022   Prob (F-statistic):                    2.06e-42
Time:                        12:15:56   Log-Likelihood:                         -2183.1
No. Observations:                 149   AIC:                                      4376.
Df Residuals:                     144   BIC:                                      4391.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Green_CNT       344.8788     55.288      6.238      0.000     235.597     454.161
Green_AMT        27.8859      2.208     12.629      0.000      23.521      32.250
Small_CNT      -609.6798     98.182     -6.210      0.000    -803.744    -415.615
Member          221.0780     28.991      7.626      0.000     163.776     278.380
Female_member  -363.3484     75.958     -4.784      0.000    -513.485    -213.211
==============================================================================
Omnibus:                      157.547   Durbin-Watson:                   1.931
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3697.776
Skew:                           3.859   Prob(JB):                         0.00
Kurtosis:                      26.153   Cond. No.                         59.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""